In [1]:
from decomp_utils import *
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
import tensorflow_probability as tfp

D:\anaconda\envs\decomp\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
path = "~"
train_ls = [1, 3]
test_ls = 2
overlap = 5 #ms
x_sEMG = 'EMGs'
y_spikes = 'Spikes'
mu = [0,1,2,3,4,5,6,7,8,9]

In [3]:
x, y = load_train(path, train_ls, overlap,mu)
x = np.expand_dims(x, axis=3)   #2D Convolution

In [4]:
select_motor = 9
y = y[select_motor]

In [5]:
input_shape = x.shape[1:]
output_shape = 1

In [6]:
#Hyper-parameter

input_shape = input_shape
patch_size = 12
no_patches = 50  # h * w / p^2
dims = 100
no_transformer_layers = 4
no_heads = 2
transformer_units = [dims * 2,dims]  # Size of the transformer layers
mlp_head_units = [240, 10]   # [2048, 1024]
num_classes = 1
learning_rate = 0.001
weight_decay = 0.0001

In [7]:
class Generate_Patches(tf.keras.layers.Layer):
    
    def __init__(self, patch_size):
        super(Generate_Patches, self).__init__()
        self.patch_size = patch_size
    
    
    def call(self, images):
        
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(images=images,
                           sizes=[1, self.patch_size, self.patch_size, 1],
                           strides=[1, self.patch_size, self.patch_size, 1],
                           rates=[1, 1, 1, 1],
                           padding='VALID')
        h_w_c = patches.shape[-1]
        patch_reshape = tf.reshape(patches, [batch_size, -1, h_w_c]) # (batch_size, no_of_patches, h*w*c)
        
        return patch_reshape

In [8]:
class Embed_Position(tf.keras.layers.Layer):
    
    def __init__(self, num_patches, projection_dims):
        
        super(Embed_Position, self).__init__()
        
        self.num_patches = num_patches
        self.project = tf.keras.layers.Dense(units=projection_dims)
        self.embed = tf.keras.layers.Embedding(input_dim=num_patches, output_dim=projection_dims)
    
    def call(self, patch):
        
        position = tf.range(start=0, limit=self.num_patches, delta=1)
        encode = self.project(patch) + self.embed(position)
        
        return encode

In [9]:
def mlp_mixer(inputs, no_units, drop_out):
    
    for units in no_units:
        x = tf.keras.layers.Dense(units=units, activation = tf.nn.gelu)(inputs)
        x = tf.keras.layers.Dropout(drop_out)(x)
        # x = tf.keras.layers.Dense(inputs.shape[-1], activation=tf.nn.gelu)(x)
    
    return x

In [10]:
def vit_trans_decomp():
    
    inputs = tf.keras.Input(shape=input_shape)
    #x = tf.keras.layers.BatchNormalization()(inputs)
    returned_patches = Generate_Patches(patch_size)(inputs)
    encoded_patches = Embed_Position(no_patches, dims)(returned_patches)
    #encoded_patches = tf.keras.layers.BatchNormalization()(encoded_patches)
    # Create multiple layers of the Transformer block.
    for _ in range(no_transformer_layers):
        # Layer normalization 1.
        input_two_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = tf.keras.layers.MultiHeadAttention(num_heads=no_heads, key_dim=dims, dropout=0.1)(input_two_attention, input_two_attention)
        # Skip connection 1.
        input_two_attention_2 = tf.keras.layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        input_two_attention_3  = tf.keras.layers.LayerNormalization(epsilon=1e-6)(input_two_attention_2)
        # MLP.
        input_two_attention_3 = mlp_mixer(input_two_attention_3, no_units=transformer_units, drop_out=0.1)
        # Skip connection 2.
        encoded_patches = tf.keras.layers.Add()([input_two_attention_3, input_two_attention_3])
        
        
# Create a [batch_size, projection_dim] tensor.
    representation = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = tf.keras.layers.Flatten()(representation)
    representation = tf.keras.layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp_mixer(representation, no_units=mlp_head_units, drop_out=0.5)
    # Classify outputs.
    logits = tf.keras.layers.Dense(1, activation='sigmoid')(features)
    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs, outputs=logits, name="sEMG-Decomposition")
    
    return model

In [11]:
model = vit_trans_decomp()
model.summary()

Model: "sEMG-Decomposition"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 generate__patches (Generate_Pa  (None, None, 144)   0           ['input_1[0][0]']                
 tches)                                                                                           
                                                                                                  
 embed__position (Embed_Positio  (None, 50, 100)     19500       ['generate__patches[0][0]']      
 n)                                                                              

 dropout_7 (Dropout)            (None, 50, 100)      0           ['dense_8[0][0]']                
                                                                                                  
 add_7 (Add)                    (None, 50, 100)      0           ['dropout_7[0][0]',              
                                                                  'dropout_7[0][0]']              
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 50, 100)     200         ['add_7[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 flatten (Flatten)              (None, 5000)         0           ['layer_normalization_8[0][0]']  
                                                                                                  
 dropout_8

In [12]:
accuracy_callback = AccuracyCallback('accuracy')
f1_callback = AccuracyCallback('f1_m')


n_batch = 128
n_epochs = 3
ls =  'binary_crossentropy'
mtr = ['mse', 'accuracy',  f1_m]
opt = 'rmsprop'
model.compile(optimizer=opt, loss=ls, metrics=mtr)

In [13]:
history = model.fit(x,y, validation_split=0.2, batch_size=n_batch, epochs=n_epochs)

Epoch 1/3
129/129 [==============================] - 15s 66ms/step - loss: 0.1783 - mse: 0.0358 - accuracy: 0.9738 - f1_m: 0.0100 - val_loss: 0.0775 - val_mse: 0.0121 - val_accuracy: 0.9879 - val_f1_m: 0.0000e+00
Epoch 2/3
129/129 [==============================] - 7s 54ms/step - loss: 0.1538 - mse: 0.0296 - accuracy: 0.9844 - f1_m: 0.0022 - val_loss: 0.1020 - val_mse: 0.0121 - val_accuracy: 0.9879 - val_f1_m: 0.0000e+00
Epoch 3/3
129/129 [==============================] - 7s 52ms/step - loss: 0.1423 - mse: 0.0274 - accuracy: 0.9854 - f1_m: 0.0000e+00 - val_loss: 0.0879 - val_mse: 0.0121 - val_accuracy: 0.9879 - val_f1_m: 0.0000e+00


In [14]:
# Test Validation

x_ts, y_ts = load_test(path, test_ls, overlap,mu)
x_ts = np.expand_dims(x_ts, axis=3)

In [15]:
model.evaluate(x_ts, y_ts)

319/319 [==============================] - 6s 17ms/step - loss: 0.2022 - mse: 0.0281 - accuracy: 0.9718 - f1_m: 0.0000e+00


[0.20221994817256927, 0.028120050206780434, 0.9718213081359863, 0.0]

In [16]:
pred = model.predict(x_ts)
y_pred = np.argmax(pred, axis=-1)
confusion = confusion_matrix(y_ts[select_motor], y_pred)

319/319 [==============================] - 4s 11ms/step


In [17]:
TP = confusion[0, 0]
TN = confusion[1, 1]
FP = confusion[1, 0]
FN = confusion[0, 1]

In [18]:
precision = TP / float(TP + FP)
print("Precision is {}".format(precision))

Precision is 0.9919489445262641


In [19]:
recall = TP / float(TP + FN)
print("recall is {}".format(recall))

#RECALL SAME AS SENSITIVITY

recall is 1.0
